In [9]:
import pandas as pd


I part 2 starter vi ud med at indlæse vores 'train_set.csv', 'validation_set.csv' og 'test_set.csv'

In [10]:
train_data = pd.read_csv('train_set.csv')
val_data = pd.read_csv('validation_set.csv')
test_data = pd.read_csv('test_set.csv')

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

# Indlæs træningsdata
train_data = pd.read_csv('train_set.csv')

# Erstat NaN-værdier i 'stemmed_content' med en tom streng
train_data['stemmed_content'].fillna('', inplace=True)

# Definer dine kategorier
reliable_categories = ['reliable', 'political']
fake_categories = ['fake', 'bias', 'conspiracy', 'hate', 'junksci', 'rumor', 'unreliable', 'satire']

# Opret en funktion, der omdanner 'type' til binære værdier
def map_to_binary(article_type):
    if article_type in reliable_categories:
        return 1  # 'reliable'
    elif article_type in fake_categories:
        return 0  # 'fake'# Sikrer, at funktionen returnerer 0 for alle andre tilfælde, hvilket eliminerer risikoen for NaN-værdier
    return 0  

# Anvend denne funktion på dit datasæt for at oprette en binær label
y_train = train_data['type'].apply(map_to_binary)

# Nu er 'y_train' klar uden risiko for NaN-værdier
X_train = train_data['stemmed_content']  # 'X_train' er også klar til brug

# Opret en pipeline med TF-IDF og Naive Bayes
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('classifier', MultinomialNB())
])

# Træn modellen på træningsdata
pipeline.fit(X_train, y_train)

# Her er den del hvor vi indlæser valideringsdata og forudsiger
# Antager at du har en 'val_set.csv' som valideringsdatasæt
val_data = pd.read_csv('validation_set.csv')
val_data['stemmed_content'].fillna('', inplace=True)
X_val = val_data['stemmed_content']
y_val = val_data['type'].apply(map_to_binary)

# Brug pipelinen til at forudsige på valideringsdata
y_pred = pipeline.predict(X_val)

# Beregn og udskriv præcision for valideringsdata
val_accuracy = accuracy_score(y_val, y_pred)
print(f"Validation accuracy: {val_accuracy}")

Validation accuracy: 0.8614271356783919
LogisticRegression MSE: 0.0957239303443731


Iforhold til meta data overvejede vi domain men i og med vi tidlgiere fandt at hver domain kun udgiver 1 type artikel mener vi at det vil "overfitte" vores model, hvor vores model bare genkender domain istedet for at forudsige. 

skriv i rapporten omat vi har prøvet at bruge meta feature data som domain og author hvor vi feks så overfitting med domain f.eks. pog noget andet at skrive om...

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score
import pandas as pd

# Antager at train_data og val_data allerede er indlæst

# Forberedelse af data (Erstat NaN-værdier og anvend map_to_binary funktionen)
train_data['stemmed_content'].fillna('', inplace=True)
train_data['authors'].fillna('', inplace=True)
val_data['stemmed_content'].fillna('', inplace=True)
val_data['authors'].fillna('', inplace=True)

y_train = train_data['type'].apply(map_to_binary)
y_val = val_data['type'].apply(map_to_binary)

# Forbered X_train og X_val som dictionaries til ColumnTransformer
X_train = train_data[['stemmed_content', 'authors']]
X_val = val_data[['stemmed_content', 'authors']]

# Opret en ColumnTransformer til at forarbejde de forskellige kolonner
preprocessor = ColumnTransformer(
    transformers=[
        ('text', TfidfVectorizer(), 'stemmed_content'),
        ('authors', OneHotEncoder(handle_unknown='ignore'), ['authors'])
    ]
)

# Opret en pipeline med preprocessor og Naive Bayes
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', MultinomialNB())
])

# Træn modellen på træningsdata
pipeline.fit(X_train, y_train)

# Forudsige på valideringsdata
y_pred = pipeline.predict(X_val)

# Beregn og udskriv præcision for valideringsdata
val_accuracy = accuracy_score(y_val, y_pred)
print(f"Validation accuracy: {val_accuracy}")


Validation accuracy: 0.8987537688442211


Part 3


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import precision_score, recall_score, f1_score


# Indlæs træningsdata

# Erstat NaN-værdier i 'stemmed_content' med en tom streng
train_data['stemmed_content'].fillna('', inplace=True)

# Definer dine kategorier
reliable_categories = ['reliable', 'political']
fake_categories = ['fake', 'conspiracy', 'hate', 'junksci', 'rumor', 'unreliable', 'satire','bias']

# Opret en funktion, der omdanner 'type' til binære værdier
def map_to_binary(article_type):
    if article_type in reliable_categories:
        return 1  # 'reliable'
    elif article_type in fake_categories:
        return 0  # 'fake'
    # Sikrer, at funktionen returnerer 0 for alle andre tilfælde, hvilket eliminerer risikoen for NaN-værdier
    return 0  

# Anvend denne funktion på dit datasæt for at oprette en binær label
y_train = train_data['type'].apply(map_to_binary)

# Nu er 'y_train' klar uden risiko for NaN-værdier
X_train = train_data['stemmed_content']  # 'X_train' er også klar til brug

# Opret en pipeline med TF-IDF og logistisk regression, øg antallet af iterationer
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('classifier', LogisticRegression(max_iter=10000))  
])

# Træn modellen på træningsdata
pipeline.fit(X_train, y_train)

# Her er den del hvor vi indlæser valideringsdata og forudsiger
# Antager at du har en 'val_set.csv' som valideringsdatasæt
val_data['stemmed_content'].fillna('', inplace=True)
X_val = val_data['stemmed_content']
y_val = val_data['type'].apply(map_to_binary)

# Brug pipeline til at forudsige på valideringsdata
y_pred = pipeline.predict(X_val)

# Forudsagte sandsynligheder (hvis du har brug for dem til MSE beregning)
y_pred_probs = pipeline.predict_proba(X_val)[:, 1]

# Beregn og udskriv præcision for valideringsdata
val_accuracy = accuracy_score(y_val, y_pred)
print(f"Validation accuracy: {val_accuracy}")

# Beregn MSE ved at sammenligne forudsagte sandsynligheder med de faktiske labels
# Bemærk: Dette er ikke standard for klassificering og anvendes normalt i regression.
mse_logistic = mean_squared_error(y_val, y_pred_probs)
print(f"LogisticRegression MSE: {mse_logistic}")

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Beregn præcision, recall og F1-score
precision = precision_score(y_val, y_pred)
recall = recall_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, mean_squared_error, precision_score, recall_score, f1_score
import pandas as pd

# Antager at train_data og val_data allerede er indlæst

# Erstat NaN-værdier i 'stemmed_content' og 'domain' med en tom streng
train_data['stemmed_content'].fillna('', inplace=True)
train_data['authors'].fillna('', inplace=True)

val_data['stemmed_content'].fillna('', inplace=True)
val_data['authors'].fillna('', inplace=True)

# Definer dine kategorier
reliable_categories = ['reliable', 'political']
fake_categories = ['fake', 'bias', 'conspiracy', 'hate', 'junksci', 'rumor', 'unreliable', 'satire']

# Opret en funktion, der omdanner 'type' til binære værdier
def map_to_binary(article_type):
    return 1 if article_type in reliable_categories else 0

# Anvend denne funktion på dit datasæt for at oprette en binær label
y_train = train_data['type'].apply(map_to_binary)
y_val = val_data['type'].apply(map_to_binary)

# Forbered features
X_train = train_data[['stemmed_content', 'authors']]
X_val = val_data[['stemmed_content', 'authors']]

# Opret en ColumnTransformer til at forarbejde de forskellige kolonner
preprocessor = ColumnTransformer(
    transformers=[
        ('text', TfidfVectorizer(), 'stemmed_content'),
        ('authors', OneHotEncoder(handle_unknown='ignore'), ['authors'])
    ]
)

# Opret en pipeline med preprocessor og logistisk regression
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000))
])

# Træn modellen på træningsdata
pipeline.fit(X_train, y_train)

# Forudsige på valideringsdata
y_pred = pipeline.predict(X_val)

# Beregn og udskriv præcision for valideringsdata
val_accuracy = accuracy_score(y_val, y_pred)
print(f"Validation accuracy: {val_accuracy}")

# Beregn MSE ved at sammenligne forudsagte sandsynligheder med de faktiske labels
y_pred_probs = pipeline.predict_proba(X_val)[:, 1]
mse_logistic = mean_squared_error(y_val, y_pred_probs)
print(f"LogisticRegression MSE: {mse_logistic}")

Iforhold til meta data overvejede vi domain men i og med vi tidlgiere fandt at hver domain kun udgiver 1 type artikel mener vi at det vil "overfitte" vores model, hvor vores model bare genkender domain istedet for at forudsige. 